In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from specutils import XraySpectrum1D, ARF, RMF

In [ ]:
from astropy.modeling.powerlaws import PowerLaw1D
import astropy.units as u
from astropy.io import fits

In [ ]:
def test_create_from_arrays():
    # Test that XraySpectrum1D can be initialized
    amp0, alpha0 = 3.e-3, 2.0
    powlaw0 = PowerLaw1D(amplitude=amp0, alpha=alpha0, x_0=1.e3)
    energy = np.linspace(0.2, 10.0, 8000)
    elo = energy[:-1] * u.keV
    ehi = energy[1:] * u.keV
    emid = 0.5 * (elo + ehi)
    counts = np.random.poisson(lam=powlaw0(emid.value), size=len(emid)) * u.ct
    test_spec = XraySpectrum1D(elo, ehi, counts, exposure=1.0*u.second)
    return test_spec

In [ ]:
ts = test_create_from_arrays()

## Try loading some example data

In [ ]:
ANGS = ['angstrom', 'Angstrom', 'angs', 'Angs', 'A']
KEV = ['keV', 'kev']

def load_hetg(filename, arf=None, rmf=None):
    ff   = fits.open(filename)
    data = ff[1].data

    bin_unit = u.Unit(data.columns['BIN_LO'].unit)
    bin_lo   = data['BIN_LO'] * bin_unit
    bin_hi   = data['BIN_HI'] * bin_unit
    
    counts   = data['COUNTS'] * u.ct
    exposure = ff[1].header['EXPOSURE'] * u.second
    ff.close()
    result = XraySpectrum1D(bin_lo, bin_hi, counts, exposure=exposure,
                            arf=arf, rmf=rmf)
    return result

In [ ]:
heg_arf = ARF.read('17392/heg_-1.arf', block='SPECRESP')

In [ ]:
heg_rmf = RMF.read('17392/heg_-1.rmf')

In [ ]:
chandra = load_hetg('17392/heg_-1.pha', arf=heg_arf, rmf=heg_rmf)

In [ ]:
def test_apply_model(test_spec):
    # Test that one can evaluate XraySpectrum1D with a model
    new_model = PowerLaw1D(amplitude=3.e-5, alpha=0.0, x_0=1.e3)
    model_flux = new_model(test_spec.spectral_axis.value) / (u.cm**2 * u.second)
    ymodel = test_spec.apply_response(model_flux)
    assert len(ymodel) == len(test_spec.spectral_axis)
    return ymodel

In [ ]:
test_model = test_apply_model(chandra)

In [ ]:
plt.plot(chandra.spectral_axis, chandra.counts)
plt.plot(chandra.spectral_axis, test_model, 'r')
plt.xlabel(chandra.spectral_axis.unit)
plt.ylabel(chandra.flux.unit)

## Try the loader

In [ ]:
chandra2 = XraySpectrum1D.read('17392/heg_-1.pha', format='chandra_hetg', 
                               arf='17392/heg_-1.arf', rmf='17392/heg_-1.rmf')

In [ ]:
plt.plot(chandra2.spectral_axis, chandra2.counts)
plt.plot(chandra2.spectral_axis, test_model, 'r')
plt.xlabel(chandra2.spectral_axis.unit)
plt.ylabel(chandra2.flux.unit)